In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.utils import all_estimators

In [5]:
#set this to the root directory of the project
path_root_dir="C:/Users/Ramesh Babu/BuzzOnEarth/datasets/"
data = pd.read_csv(path_root_dir + "processed/all_city_data_with_pop.csv")

In [6]:
data.head()

,Unnamed: 0.1,Unnamed: 0,geometry,parking,edges,EV_stations,parking_space,civic,restaurant,park,...,cinema,library,commercial,retail,townhall,government,residential,city,population,Berlin_data_onlycenter_
0,0,0,"POLYGON ((8.4727605 50.099822499999995, 8.4730...",0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
1,1,1,"POLYGON ((8.4775730092433 50.10302720327834, 8...",0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,0.000000,NaN
2,2,2,"POLYGON ((8.479750879173663 50.09863320231676,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
3,3,3,"POLYGON ((8.479688060978736 50.10443297769501,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
4,4,4,"POLYGON ((8.47965547981383 50.107440331063444,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,0.000000,NaN


In [7]:
#filtering out columsn to be used for modeling
data = data[['geometry','city','EV_stations', 'parking', 'edges',
        'parking_space', 'civic', 'restaurant', 'park', 'school',
       'node', 'Community_centre', 'place_of_worship', 'university', 'cinema',
       'library', 'commercial', 'retail', 'townhall', 'government',
       'residential', 'population']]
print("data size:" , data.shape)
data = data.dropna()
print("data size after dropping na:" , data.shape)
data.head()

data size: (10824, 22)
data size after dropping na: (10129, 22)


,geometry,city,EV_stations,parking,edges,parking_space,civic,restaurant,park,school,...,place_of_worship,university,cinema,library,commercial,retail,townhall,government,residential,population
0,"POLYGON ((8.4727605 50.099822499999995, 8.4730...",Frankfurt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,9.014051
1,"POLYGON ((8.4775730092433 50.10302720327834, 8...",Frankfurt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0.000000
2,"POLYGON ((8.479750879173663 50.09863320231676,...",Frankfurt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,9.014051
3,"POLYGON ((8.479688060978736 50.10443297769501,...",Frankfurt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,9.014051
4,"POLYGON ((8.47965547981383 50.107440331063444,...",Frankfurt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0.000000


In [8]:
def data_splitter(data, train_cities=None, test_cities=None, test_size=0.2, random_state=42):

    if train_cities is not None:
        train = data[data['city'].isin(train_cities)]
        test = data[data['city'].isin(test_cities)]


        X_train = train.drop(['city','geometry', 'EV_stations'], axis=1)
        y_train = train['EV_stations'].astype(int)
        y_train = y_train.apply(lambda x: 1 if x > 0 else 0)

        X_test = test.drop(['city','geometry', 'EV_stations'], axis=1)
        y_test = test['EV_stations'].astype(int)
        y_test = y_test.apply(lambda x: 1 if x > 0 else 0)
    else:
        X = data.drop(['city','geometry', "EV_stations"], axis=1)
        y = data['EV_stations']
        y = y.apply(lambda x: 1 if x > 0 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = data_splitter(data)

In [10]:

# logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print("Logistic Regression Test Accuracy: ", logreg.score(X_test, y_test))
# classification report
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

Logistic Regression Test Accuracy:  0.8958538993089832
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      1786
           1       0.62      0.30      0.41       240

    accuracy                           0.90      2026
   macro avg       0.77      0.64      0.68      2026
weighted avg       0.88      0.90      0.88      2026



C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Get all classification model classes
classifiers = all_estimators(type_filter='classifier')

# Initialize result table
results = []
models = {}
# Run models and collect results
for name, ClassifierClass in tqdm(classifiers):
    try:
        # Initialize model
        model = ClassifierClass()
        model.fit(X_train, y_train)
        models[name] = model
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')
        auc = roc_auc_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        
        # Append results
        results.append([name, accuracy, precision, recall, f1, auc, balanced_accuracy])
    except Exception as e:
        print(f"Error occurred for {name}: {str(e)}")

# Create a DataFrame from results
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score", "AUC", "Balanced Accuracy"])
results_df = results_df.sort_values(by=['F1-score', 'AUC'], ascending=False)
print(results_df)


  0%|          | 0/41 [00:00<?, ?it/s]

Error occurred for CategoricalNB: index 7 is out of bounds for axis 1 with size 7
Error occurred for ClassifierChain: _BaseChain.__init__() missing 1 required positional argument: 'base_estimator'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Error occurred for KNeighborsClassifier: 'Flags' object has no attribute 'c_contiguous'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\l

Error occurred for MultiOutputClassifier: MultiOutputClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for NuSVC: specified nu is infeasible
Error occurred for OneVsOneClassifier: OneVsOneClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OneVsRestClassifier: OneVsRestClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OutputCodeClassifier: OutputCodeClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for RadiusNeighborsClassifier: No neighbors found for test samples array([   0,    1,    2, ..., 2022, 2023, 2025], dtype=int64), you can try using larger radius, giving a label for outliers, or considering removing them from your dataset.
Error occurred for StackingClassifier: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
Error occurred for VotingClassifier: VotingClassifier.__init__() missing 1

In [12]:
def run_experiment(X_train, X_test, y_train, y_test):
    # Get all classification model classes
    classifiers = all_estimators(type_filter='classifier')

    # Initialize result table
    results = []
    models = {}
    # Run models and collect results
    for name, ClassifierClass in tqdm(classifiers):
        try:
            # Initialize model
            model = ClassifierClass()
            model.fit(X_train, y_train)
            models[name] = model
            y_pred = model.predict(X_test)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            recall = recall_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            auc = roc_auc_score(y_test, y_pred)
            balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
            
            # Append results
            results.append([name, accuracy, precision, recall, f1, auc, balanced_accuracy])
        except Exception as e:
            print(f"Error occurred for {name}: {str(e)}")

    # Create a DataFrame from results
    results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score", "AUC", "Balanced Accuracy"])
    results_df = results_df.sort_values(by=['F1-score', 'AUC'], ascending=False)
    return results_df, models


In [13]:
result_df, models = run_experiment(X_train, X_test, y_train, y_test)

  0%|          | 0/41 [00:00<?, ?it/s]

Error occurred for CategoricalNB: index 7 is out of bounds for axis 1 with size 7
Error occurred for ClassifierChain: _BaseChain.__init__() missing 1 required positional argument: 'base_estimator'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Error occurred for KNeighborsClassifier: 'Flags' object has no attribute 'c_contiguous'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\l

Error occurred for MultiOutputClassifier: MultiOutputClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for NuSVC: specified nu is infeasible
Error occurred for OneVsOneClassifier: OneVsOneClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OneVsRestClassifier: OneVsRestClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OutputCodeClassifier: OutputCodeClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for RadiusNeighborsClassifier: No neighbors found for test samples array([   0,    1,    2, ..., 2022, 2023, 2025], dtype=int64), you can try using larger radius, giving a label for outliers, or considering removing them from your dataset.
Error occurred for StackingClassifier: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
Error occurred for VotingClassifier: VotingClassifier.__init__() missing 1

In [14]:
results_df

,Model,Accuracy,Precision,Recall,F1-score,AUC,Balanced Accuracy
12,HistGradientBoostingClassifier,0.900296,0.767657,0.705403,0.730580,0.705403,0.705403
0,AdaBoostClassifier,0.900296,0.768031,0.703600,0.729434,0.703600,0.703600
11,GradientBoostingClassifier,0.899309,0.766435,0.694023,0.722009,0.694023,0.694023
25,RandomForestClassifier,0.903258,0.785986,0.685442,0.720676,0.685442,0.685442
19,MLPClassifier,0.900790,0.776544,0.680436,0.714209,0.680436,0.680436
15,LinearDiscriminantAnalysis,0.894373,0.750447,0.680403,0.707106,0.680403,0.680403
8,ExtraTreesClassifier,0.898815,0.770472,0.672102,0.705814,0.672102,0.672102
1,BaggingClassifier,0.891905,0.742488,0.671790,0.698280,0.671790,0.671790
9,GaussianNB,0.866239,0.687214,0.707727,0.696568,0.707727,0.707727
20,MultinomialNB,0.872162,0.693752,0.693052,0.693401,0.693052,0.693052


In [16]:
results_df.to_csv("../results/all_cities_random_shuffle.csv", index=False)

In [17]:
"""
Berlin, Munich, Stuttgart, Frankfurt: Big CITY EXP-1
Kalsruhe, trier, saarbrucken, mainz: EXP-2
"""

# EXP-1
big_cities = ['Berlin', 'Munich', 'Stuttgart', 'Frankfurt']
small_cities = ['Karlsruhe', 'Trier', 'Saarbrücken', 'Mainz']


# make a table in the end to summarise the results of all experiments

# big cities splited in trian and test where only one big city is test and all possible combinations for this
for city in tqdm(big_cities):
    test_cities = [city]
    train_cities = [x for x in big_cities if x != city]
    X_train, X_test, y_train, y_test = data_splitter(data, train_cities=train_cities, test_cities=test_cities)
    results_df, models = run_experiment(X_train, X_test, y_train, y_test)
    results_df.to_csv(f"../results/big_cities_test_city_{city}_.csv", index=False)
    



# small cities splited in trian and test where only one small city is test and all possible combinations for this
for city in tqdm(small_cities):
    test_cities = [city]
    train_cities = [x for x in small_cities if x != city]
    X_train, X_test, y_train, y_test = data_splitter(data, train_cities=train_cities, test_cities=test_cities)
    results_df, models = run_experiment(X_train, X_test, y_train, y_test)
    results_df.to_csv(f"../results/small_cities_test_city_{city}_.csv", index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

Error occurred for CategoricalNB: index 51 is out of bounds for axis 1 with size 51
Error occurred for ClassifierChain: _BaseChain.__init__() missing 1 required positional argument: 'base_estimator'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Error occurred for KNeighborsClassifier: 'Flags' object has no attribute 'c_contiguous'


...

C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Error occurred for KNeighborsClassifier: 'Flags' object has no attribute 'c_contiguous'


C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\semi_supervised\_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\l

Error occurred for MultiOutputClassifier: MultiOutputClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for NuSVC: specified nu is infeasible
Error occurred for OneVsOneClassifier: OneVsOneClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OneVsRestClassifier: OneVsRestClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for OutputCodeClassifier: OutputCodeClassifier.__init__() missing 1 required positional argument: 'estimator'
Error occurred for RadiusNeighborsClassifier: No neighbors found for test samples array([  0,   2,   3,   8,  11,  12,  14,  16,  17,  19,  21,  22,  24,
        26,  27,  28,  29,  30,  33,  35,  36,  37,  38,  39,  40,  43,
        44,  45,  46,  47,  48,  49,  50,  54,  56,  57,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  72,  73,  74,  75,  77,
        78,  79,  80,  81,  82,  83,  84,  86,  87,  89,  90,  91,  92,
        9

C:\Users\Ramesh Babu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
results_df

,Model,Accuracy,Precision,Recall,F1-score,AUC,Balanced Accuracy
2,BernoulliNB,0.847561,0.633567,0.812410,0.665443,0.812410,0.812410
28,SGDClassifier,0.939024,0.859903,0.597812,0.643133,0.597812,0.597812
20,MultinomialNB,0.894309,0.626664,0.652892,0.637969,0.652892,0.652892
17,LogisticRegression,0.912602,0.651632,0.623163,0.635386,0.623163,0.623163
8,ExtraTreesClassifier,0.939024,0.898675,0.584620,0.626934,0.584620,0.584620
4,ComplementNB,0.880081,0.605113,0.645233,0.620127,0.645233,0.645233
25,RandomForestClassifier,0.934959,0.803313,0.582432,0.619342,0.582432,0.582432
16,LinearSVC,0.790650,0.606883,0.808159,0.617548,0.808159,0.808159
0,AdaBoostClassifier,0.922764,0.676409,0.589059,0.614229,0.589059,0.589059
22,PassiveAggressiveClassifier,0.939024,0.969199,0.571429,0.609110,0.571429,0.571429


In [19]:
import pandas as pd
import glob

# Get a list of all result files from different experiments
result_files = glob.glob("../results/*.csv")

# Create a dictionary to store the total AUC and count for each model
auc_sum_per_model = {}
count_per_model = {}

# Iterate over each result file
for file in result_files:
    print(file)
    # Load the results for each experiment
    results = pd.read_csv(file)
    
    # Iterate over each row in the results
    for _, row in results.iterrows():
        model = row['Model']
        auc = row['AUC']
        
        # Update the total AUC and count for the model
        if model in auc_sum_per_model:
            auc_sum_per_model[model] += auc
            count_per_model[model] += 1
        else:
            auc_sum_per_model[model] = auc
            count_per_model[model] = 1

# Calculate the average AUC for each model
average_auc_per_model = {model: auc_sum_per_model[model] / count_per_model[model] for model in auc_sum_per_model}

# Create a DataFrame from the average AUC dictionary
average_auc_df = pd.DataFrame(list(average_auc_per_model.items()), columns=['Model', 'Average AUC'])

# Sort the DataFrame by Average AUC in descending order
sorted_models = average_auc_df.sort_values(by='Average AUC', ascending=False)

# Select the top 5 models
top_5_models = sorted_models.head(5)

# Display the best models
print(top_5_models)


../results\all_cities_random_shuffle.csv
../results\big_cities_test_city_Berlin_.csv
../results\big_cities_test_city_Frankfurt_.csv
../results\big_cities_test_city_Munich_.csv
../results\big_cities_test_city_Stuttgart_.csv
../results\small_cities_test_city_Karlsruhe_.csv
../results\small_cities_test_city_Mainz_.csv
../results\small_cities_test_city_Saarbrücken_.csv
../results\small_cities_test_city_Trier_.csv
              Model  Average AUC
12      BernoulliNB     0.779203
16  NearestCentroid     0.766557
10     ComplementNB     0.671147
9     MultinomialNB     0.662987
25    SGDClassifier     0.658037


In [20]:
top_5_models

,Model,Average AUC
12,BernoulliNB,0.779203
16,NearestCentroid,0.766557
10,ComplementNB,0.671147
9,MultinomialNB,0.662987
25,SGDClassifier,0.658037


In [24]:
import pandas as pd
import glob

# Get a list of all result files from different experiments
result_files = glob.glob("../results/*.csv")

# Create an empty DataFrame to store the combined results
combined_results = pd.DataFrame()

# Create an empty DataFrame to store the summary
summary_results = pd.DataFrame(columns=['type_city'])

# Iterate over each result file
for type_city in ['big', 'small', 'all']:
    # Reset combined_results for each type_city iteration
    combined_results = pd.DataFrame()

    # List to store dataframes to be concatenated later
    data_frames = []

    # Iterate over each result file
    for file in result_files:
        # Load the results for each experiment
        if type_city in file:
            results = pd.read_csv(file)
            
            # Append the results to the list of dataframes
            data_frames.append(results)

    # Concatenate all dataframes in the list
    if data_frames:
        combined_results = pd.concat(data_frames)

    # Calculate the average AUC for each model
    average_auc_per_model = combined_results.groupby('Model')['AUC'].mean()
    
    # Sort the models by average AUC in descending order
    sorted_models = average_auc_per_model.sort_values(ascending=False)
    
    # Get the top 5 models
    top_5_models = sorted_models.head(5).index.tolist()

    # Filter the results to include only the rows corresponding to the top 5 models
    filtered_results = combined_results[combined_results['Model'].isin(top_5_models)]

    # Calculate the average AUC for each model
    average_auc_by_model = filtered_results.groupby('Model')['AUC'].mean()
    
    # Create a row with type_city and average AUC values for each model
    row = {'type_city': type_city}
    row.update(average_auc_by_model.to_dict())
    
    # Append the row to the summary_results DataFrame
    summary_results = pd.concat([summary_results, pd.DataFrame([row])], ignore_index=True)

# Display the summary_results DataFrame
print(summary_results)


  type_city  BernoulliNB  ComplementNB  GaussianNB  NearestCentroid   
0       big     0.764115      0.669090    0.660482         0.715620  \
1     small     0.810223      0.654498         NaN         0.853362   
2       all     0.799321      0.673888         NaN         0.834474   

   QuadraticDiscriminantAnalysis  LinearSVC  MultinomialNB  SGDClassifier  
0                       0.657869        NaN            NaN            NaN  
1                            NaN   0.685329       0.666221            NaN  
2                            NaN        NaN       0.675165       0.676748  


In [25]:
summary_results

,type_city,BernoulliNB,ComplementNB,GaussianNB,NearestCentroid,QuadraticDiscriminantAnalysis,LinearSVC,MultinomialNB,SGDClassifier
0,big,0.764115,0.669090,0.660482,0.715620,0.657869,NaN,NaN,NaN
1,small,0.810223,0.654498,NaN,0.853362,NaN,0.685329,0.666221,NaN
2,all,0.799321,0.673888,NaN,0.834474,NaN,NaN,0.675165,0.676748


In [26]:
import pandas as pd
import glob

# Get a list of all result files from different experiments
result_files = glob.glob("../results/*.csv")

# Create an empty DataFrame to store the summary
summary_results = pd.DataFrame(columns=['type_city', 'AUC', 'Accuracy', 'Precision', 'Recall'])

# Iterate over each type of city
for type_city in ['big', 'small', 'all']:
    # List to store the DataFrames for each file of the current type_city
    data_frames = []

    # Iterate over each result file
    for file in result_files:
        # Load the results for each experiment
        if type_city in file:
            results = pd.read_csv(file)
            # Add to the list of dataframes to be concatenated
            data_frames.append(results)

    # Concatenate all dataframes into one DataFrame if data exists
    if data_frames:
        combined_results = pd.concat(data_frames)

        # Filter the results to include only the rows corresponding to the top 5 models
        filtered_results = combined_results[combined_results['Model'].isin(top_5_models)]

        # Calculate the average values for each metric (AUC, Accuracy, Precision, Recall)
        average_metrics_per_model = filtered_results.groupby('Model')[['AUC', 'Accuracy', 'Precision', 'Recall']].mean()

        # Calculate the mean of the average metrics across the top 5 models
        average_values = average_metrics_per_model.mean()

        # Create a row with type_city and the average metric values
        row = {'type_city': type_city}
        for metric in ['AUC', 'Accuracy', 'Precision', 'Recall']:
            row[metric] = average_values[metric]

        # Append the row to the summary_results DataFrame
        summary_results = pd.concat([summary_results, pd.DataFrame([row])], ignore_index=True)

# Display the summary_results DataFrame
print(summary_results)


  type_city       AUC  Accuracy  Precision    Recall
0       big  0.689336  0.819802   0.662562  0.689336
1     small  0.722180  0.887707   0.654480  0.722180
2       all  0.731919  0.860643   0.656496  0.731919


In [27]:
top_5_models

['NearestCentroid',
 'BernoulliNB',
 'SGDClassifier',
 'MultinomialNB',
 'ComplementNB']

In [28]:
summary_results.to_csv("../results/summary_results.csv", index=False)